In [1]:
from textgenrnn import textgenrnn
import pandas
import numpy as np
import rhymes as r
import importlib
importlib.reload(r)
import re

models = []

models.append(textgenrnn('weights/intro_model_weights.hdf5',
                                vocab_path='weights/intro_model_vocab.json',
                                config_path='weights/intro_model_config.json'))

models.append(textgenrnn('weights/chorus_model_weights.hdf5',
                                vocab_path='weights/chorus_model_vocab.json',
                                config_path='weights/chorus_model_config.json'))

models.append(textgenrnn('weights/verse_model_weights.hdf5',
                                vocab_path='weights/verse_model_vocab.json',
                                config_path='weights/verse_model_config.json'))

models.append(textgenrnn('weights/bridge_model_weights.hdf5',
                                vocab_path='weights/bridge_model_vocab.json',
                                config_path='weights/bridge_model_config.json'))

models.append(textgenrnn('weights/outro_model_weights.hdf5',
                                vocab_path='weights/outro_model_vocab.json',
                                config_path='weights/outro_model_config.json'))



Using TensorFlow backend.


In [11]:
#uses model to generate a list of lyrics
def gen_lyrics(model):

    lyrics = model.generate(60, temperature=1.1, max_gen_length=1000, return_as_list=True)
    
    return lyrics

#Input: A list of generated lines
#Output: dictionary where the keys are the rhymes and the values are a list of lines that rhyme
def post_process(lines):

    rhyme_dict = dict()

    #Generate of list of last words from each line
    for index, line in enumerate(lines):
        if (len(line) == 0):
            continue
        try:
            wordsInLine = line.split()
            curr_word = ""
            wordsInLine[-1] = wordsInLine[-1].lower()
            #remove puncuation
            for c in wordsInLine[-1]:
                if ord(c) >= 97 and ord(c) <= 122:
                    curr_word += c
            #only add words (remove any empty strings)
            if curr_word != "":
                IT_RHYMES_BABYYY = False
                for key in rhyme_dict.keys():
                    if (r.doTheyRhyme(curr_word, key)):
                        rhyme_dict[key].append(index)
                        IT_RHYMES_BABYYY = True
                if (not IT_RHYMES_BABYYY):
                    rhyme_dict[curr_word] = [index]
        except:
            print (curr_word, 'was not found in the dictionary')
            continue

    return rhyme_dict

#Input dictionary of rhymes with their corresponding line number
#Output: sorts each list of lines in the dictionary
def sort_dict(rhymes_dict):
    
    #loop through each key
    for key in rhymes_dict.keys():
        #get the list of lines associated with the rhyme
        index_list = rhymes_dict[key]
        length = len(index_list)
        if (length < 2):
            continue
        
        #get a list of each distance between the list of lines
        distances = []
        distances.append(index_list[1] - index_list[0])
        for i in range(1, length - 1):
            distances.append(min(index_list[i] - index_list[i - 1], index_list[i + 1] - index_list[i]))

        distances.append(index_list[length - 1] - index_list[length - 2])

        
        #do some cool sorting stuff where you sort distances and the list of lines in parallel
        distances, index_list = zip(*sorted(zip(distances, index_list)))
        distances, index_list = (list(t) for t in zip(*sorted(zip(distances, index_list))))
        
        #add the sorted list back to the dictionary
        rhymes_dict[key] = index_list
        
#Input dictionary of rhymes with their corresponding line number
#Output: returns a dictionary with rhymes that are one away    
def rhymes_one_apart(rhymes_dict):
    
    removeKeys = []
    
    #loop through each key, keep track of ones to remove
    for key in rhymes_dict.keys():
        #get the list of lines associated with the rhyme
        index_list = rhymes_dict[key]
        length = len(index_list)
        
        if (length < 2):
            removeKeys.append(key)
            continue

        new_list = []
        oneApart = False
        
        #loop through list of rhymes and only keep keys that have rhymes one line apart
        for i in range(1, length - 1):
            if (index_list[i + 1] - index_list[i] == 1 or index_list[i] - index_list[i - 1] == 1):
                oneApart = True
                new_list.append(index_list[i])
                
        #Check front and end cases
        if (index_list[1] - index_list[0] == 1):
            new_list.append(index_list[0])
        if (index_list[length - 1] - index_list[length - 2] == 1):
            new_list.append(index_list[length - 1])
        
        #add new list to key or remove the key if there were no rhymes one apart
        if (oneApart):
            rhymes_dict[key] = new_list
        else:
            removeKeys.append(key)
            
    for key in removeKeys:
        rhymes_dict.pop(key)
        
                


#Uses dictionary of rhyming lines to generate lyrics with a user's rhyming sequence
def out1(lines, rhyme_dictionary):
    print ('What rhyming scheme would you like to use? (A to Z)')
    rhyme_scheme = input()
    if (not re.fullmatch('[A-Z]+', rhyme_scheme)):
        print ('That is not a valid rhyming scheme.')
        exit()

    # count the required number of lines for each letter/section of the rhyming scheme
    rhyme_counts = dict()
    for char in rhyme_scheme:
        if char in rhyme_counts:
            rhyme_counts[char] += 1
        else:
            rhyme_counts[char] = 1
    print (rhyme_counts)

    # map each letter/section of the rhyming scheme to a rhyme in rhyme_dict
    used_rhymes = set()
    letter_to_rhyme = dict()
    for letter in rhyme_counts.keys():
        for rhyme in rhyme_dictionary.keys():
            # if the current rhyme has enough rhymes for the current letter in the rhyming scheme
            # AND the rhyme hasn't been used, use it
            if (len(rhyme_dictionary[rhyme]) >= rhyme_counts[letter] and rhyme not in used_rhymes):
                letter_to_rhyme[letter] = rhyme_dictionary[rhyme]
                used_rhymes.add(rhyme)
                break

    print (letter_to_rhyme)

    # print the results!
    rhyme_indices = dict()
    for char in rhyme_scheme:
        if (char not in rhyme_indices):
            print (lines[letter_to_rhyme[char][0]])
            rhyme_indices[char] = 1
        else:
            print (lines[letter_to_rhyme[char][rhyme_indices[char]]])
            rhyme_indices[char] += 1

In [ ]:
lines = gen_lyrics(models[1])
lines2 = []
for line in lines:
    split = line.splitlines()
    for l in split:
        if (len(l.split()) > 1):
            lines2.append(l)

In [ ]:
rhymes_dict = post_process(lines2)
sort_dict(rhymes_dict)

In [ ]:
#rhymes_one_apart(rhymes_dict)
out1(lines2, rhymes_dict)

In [12]:
### Generates entire song at once
### Takes around 4 minutes

model_lines = []

for model in models:
    lines = gen_lyrics(model)
    lines2 = []
    for line in lines:
        split = line.splitlines()
        for l in split:
            if (len(l.split()) > 1):
                lines2.append(l)
                
    model_lines.append(lines2)
    
model_dicts = []
for lines in model_lines:
    rhymes_dict = post_process(lines)
    sort_dict(rhymes_dict)
    model_dicts.append(rhymes_dict)
    print ("Done post-processing model")
    
for index, rhymes_dict in enumerate(model_dicts):
    rhymes_one_apart(rhymes_dict)
    if len(rhymes_dict) != 0:
        print("Model number", index)
        out1(model_lines[index], rhymes_dict)



  0%|                                                                                           | 0/60 [00:00<?, ?it/s]

  2%|█▍                                                                                 | 1/60 [00:00<00:47,  1.24it/s]

  3%|██▊                                                                                | 2/60 [00:01<00:37,  1.53it/s]

  5%|████▏                                                                              | 3/60 [00:01<00:35,  1.61it/s]

  7%|█████▌                                                                             | 4/60 [00:02<00:37,  1.48it/s]

  8%|██████▉                                                                            | 5/60 [00:03<00:35,  1.53it/s]

 10%|████████▎                                                                          | 6/60 [00:03<00:32,  1.69it/s]

 12%|█████████▋                                                                         | 7/60 [00:03<00:26,  1.98it/s]

 13%|███████████              

 18%|███████████████                                                                   | 11/60 [00:11<01:06,  1.36s/it]

 20%|████████████████▍                                                                 | 12/60 [00:11<00:51,  1.08s/it]

 22%|█████████████████▊                                                                | 13/60 [00:14<01:08,  1.45s/it]

 23%|███████████████████▏                                                              | 14/60 [00:14<00:54,  1.18s/it]

 25%|████████████████████▌                                                             | 15/60 [00:16<01:03,  1.41s/it]

 27%|█████████████████████▊                                                            | 16/60 [00:17<00:53,  1.23s/it]

 28%|███████████████████████▏                                                          | 17/60 [00:18<00:48,  1.12s/it]

 30%|████████████████████████▌                                                         | 18/60 [00:18<00:35,  1.18it/s]

 32%|█████████████████████████▉ 

 28%|███████████████████████▏                                                          | 17/60 [00:22<00:54,  1.26s/it]

 30%|████████████████████████▌                                                         | 18/60 [00:24<01:03,  1.51s/it]

 32%|█████████████████████████▉                                                        | 19/60 [00:25<00:49,  1.22s/it]

 33%|███████████████████████████▎                                                      | 20/60 [00:25<00:40,  1.01s/it]

 35%|████████████████████████████▋                                                     | 21/60 [00:26<00:31,  1.22it/s]

 37%|██████████████████████████████                                                    | 22/60 [00:26<00:27,  1.39it/s]

 38%|███████████████████████████████▍                                                  | 23/60 [00:27<00:27,  1.32it/s]

 40%|████████████████████████████████▊                                                 | 24/60 [00:27<00:24,  1.45it/s]

 42%|███████████████████████████

 40%|████████████████████████████████▊                                                 | 24/60 [00:31<00:33,  1.08it/s]

 42%|██████████████████████████████████▏                                               | 25/60 [00:32<00:33,  1.05it/s]

 43%|███████████████████████████████████▌                                              | 26/60 [00:32<00:26,  1.31it/s]

 45%|████████████████████████████████████▉                                             | 27/60 [00:33<00:24,  1.32it/s]

 47%|██████████████████████████████████████▎                                           | 28/60 [00:35<00:38,  1.19s/it]

 48%|███████████████████████████████████████▋                                          | 29/60 [00:37<00:46,  1.50s/it]

 50%|█████████████████████████████████████████                                         | 30/60 [00:38<00:39,  1.31s/it]

 52%|██████████████████████████████████████████▎                                       | 31/60 [00:40<00:45,  1.57s/it]

 53%|███████████████████████████

 50%|█████████████████████████████████████████                                         | 30/60 [00:42<00:24,  1.25it/s]

 52%|██████████████████████████████████████████▎                                       | 31/60 [00:44<00:34,  1.20s/it]

 53%|███████████████████████████████████████████▋                                      | 32/60 [00:46<00:39,  1.43s/it]

 55%|█████████████████████████████████████████████                                     | 33/60 [00:48<00:45,  1.67s/it]

 57%|██████████████████████████████████████████████▍                                   | 34/60 [00:49<00:38,  1.47s/it]

 58%|███████████████████████████████████████████████▊                                  | 35/60 [00:52<00:47,  1.89s/it]

 60%|█████████████████████████████████████████████████▏                                | 36/60 [00:54<00:45,  1.91s/it]

 62%|██████████████████████████████████████████████████▌                               | 37/60 [00:54<00:31,  1.38s/it]

 63%|███████████████████████████

tlc was not found in the dictionary
remix was not found in the dictionary
spiacente was not found in the dictionary
wheeooh was not found in the dictionary
themeyelashes was not found in the dictionary
estoy was not found in the dictionary
austa was not found in the dictionary
kumpent was not found in the dictionary
kumpent was not found in the dictionary
cardi was not found in the dictionary
yeahyeah was not found in the dictionary
cualquier was not found in the dictionary
bangerz was not found in the dictionary
Done post-processing model
ayy was not found in the dictionary
mmm was not found in the dictionary
yeahchorus was not found in the dictionary
inyour was not found in the dictionary
uhoh was not found in the dictionary
touchin was not found in the dictionary
ayy was not found in the dictionary
papi was not found in the dictionary
herefor was not found in the dictionary
waitin was not found in the dictionary
livin was not found in the dictionary
Done post-processing model
homies

In [9]:
for index, rhymes_dict in enumerate(model_dicts):
    rhymes_one_apart(rhymes_dict)
    if len(rhymes_dict) != 0:
        out1(model_lines[index], rhymes_dict)

What rhyming scheme would you like to use? (A to Z)
AA
{'A': 2}
{'A': [84, 94, 95, 83]}
four fire you are you can show you
i feel like i cant rhyme through
What rhyming scheme would you like to use? (A to Z)
AABB
{'A': 2, 'B': 2}
{'A': [107, 106], 'B': [41, 40]}
you the and and you
gonna and the and and you you
, at
you all i me t stack
What rhyming scheme would you like to use? (A to Z)
AABB
{'A': 2, 'B': 2}
{'A': [3, 2], 'B': [88, 87]}
at at
look the bad
i and i i i i i and i i i i i i i i i i
i i


In [13]:
for line in model_lines[1]:
    print(line)

cause baby, you look so much
so far, but can t do it all is enough
you always got this for the reason
why you got to give yourself to a lie
if that you were in the one
heal the world, catch me high
and all the kings, had the things that i had high
past it all summer
(yeah, it s the party of loving someone else
it s all the same, my love s what i love say?
even verse 1
bitch, i better
cheap ass weave
cheap ass weave
how just on fleek on you
when some cheap ass weave
and let me crawl inside beg
blah - blah - blah - blah, blah - blah
would you not for you (just let me know this is it
let me sing mad mad
you did it? so, let me get it loud? (about it?
dear has come with back
cause i slim shady, yes you up the other name, excuse me my can who can? name
name slim shady, oh the name of who?
why t have the name slim shady
lonely eyes
maybe i t deserve the words
and you try all that i want you (all i need
you try all i want, baby all i know
just maybe you try better, i just might go with skin yo

In [8]:
print(model_dicts[0])

{'no': [116, 117]}
